### Extract Data from PDF Files with Python

        El codigo para poder leer una factura espefica fué desarrollado por partes

Sample, para extrar datos contenidos una tabla del archivo PDF

In [4]:
# !pip install PDFPlumber
import pdfplumber
import pandas as pd

Para extrar todo el texto que el modulo permita leer

In [33]:
import os

root_dir = os.getcwd()

df_archivos = []
ciclo = 0

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_paths = os.path.join(dirpath, filename)

            with pdfplumber.open(file_paths) as temp:
                print(file_paths)
                ciclo += 1
                # Nombre de pdf dinamico
                print(f"pdf_{ciclo}")

                first_page = temp.pages[0]
                        # print(first_page.extract_table())
                dfnames = first_page.extract_table()

                df_t = pd.DataFrame(dfnames)
                headers = df_t.iloc[0].values
                df_t.columns = headers
                df_t.drop(index=0, axis=0, inplace=True)
                df_t = df_t.dropna()

                df_archivos.append(df_t)
                            # if ciclo == 2:
                            #     break    # break here
            df = pd.concat(df_archivos)


c:\Users\rfigu\Documents\Learn\LeerPDF\38_Septiembre_2023.pdf
pdf_1
pdf_1
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\37_Agosto_2023.pdf
pdf_2
pdf_2
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\38_Septiembre_2023.pdf
pdf_3
pdf_3
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\39_Octubre_2023.pdf
pdf_4
pdf_4


In [34]:
df

,Cod,Descripcion,Unidad de\nmedida,Cantidad,Precio\nUnitario,Descuento,Exentas,5%,10%
1,ESPERA20EX,Espera - ESPERA_EXENTA Sep/23 31:240 - Cto.:\n...,UNI,1.00,1.100.000.0,0.00,1.100.000.0,,
2,ESPERA20IM,Gastos Administrativos,UNI,1.00,275.000.0,0.00,,,275.000.0
1,ESPERA20EX,Espera - ESPERA_EXENTA Ago/23 30:240 - Cto.:\n...,UNI,1.00,1.100.000.0,0.00,1.100.000.0,,
2,ESPERA20IM,Gastos Administrativos,UNI,1.00,275.000.0,0.00,,,275.000.0
1,ESPERA20EX,Espera - ESPERA_EXENTA Sep/23 31:240 - Cto.:\n...,UNI,1.00,1.100.000.0,0.00,1.100.000.0,,
2,ESPERA20IM,Gastos Administrativos,UNI,1.00,275.000.0,0.00,,,275.000.0
1,ESPERA20EX,Espera - ESPERA_EXENTA Nov/23 33:240 - Cto.:\n...,UNI,1.00,1.140.000.0,0.00,1.140.000.0,,
2,ESPERA20IM,Gastos Administrativos,UNI,1.00,285.000.0,0.00,,,285.000.0


In [27]:
df_archivos

[          Cod                                        Descripcion  \
 1  ESPERA20EX  Espera - ESPERA_EXENTA Nov/23 33:240 - Cto.:\n...   
 2  ESPERA20IM                             Gastos Administrativos   
 
   Unidad de\nmedida Cantidad Precio\nUnitario Descuento      Exentas 5%  \
 1               UNI     1.00      1.140.000.0      0.00  1.140.000.0      
 2               UNI     1.00        285.000.0      0.00                   
 
          10%  
 1             
 2  285.000.0  ]

In [26]:
dfname

,Cod,Descripcion,Unidad de\nmedida,Cantidad,Precio\nUnitario,Descuento,Exentas,5%,10%
1,ESPERA20EX,Espera - ESPERA_EXENTA Nov/23 33:240 - Cto.:\n...,UNI,1.00,1.140.000.0,0.00,1.140.000.0,,
2,ESPERA20IM,Gastos Administrativos,UNI,1.00,285.000.0,0.00,,,285.000.0


In [16]:
dfname
df_tabla1 = pd.DataFrame(dfname)
headers = df_tabla1.iloc[0].values
df_tabla1.columns = headers
df_tabla1.drop(index=0, axis=0, inplace=True)
df_tabla1 = df_tabla1.dropna()
df_tabla1

,Cod,Descripcion,Unidad de\nmedida,Cantidad,Precio\nUnitario,Descuento,Exentas,5%,10%
1,ESPERA20EX,Espera - ESPERA_EXENTA Ago/23 30:240 - Cto.:\n...,UNI,1.00,1.100.000.0,0.00,1.100.000.0,,
2,ESPERA20IM,Gastos Administrativos,UNI,1.00,275.000.0,0.00,,,275.000.0


In [10]:
for fila in tabla:
    print(fila)

['Cod', 'Descripcion', 'Unidad de\nmedida', 'Cantidad', 'Precio\nUnitario', 'Descuento', 'Exentas', '5%', '10%']
['ESPERA20EX', 'Espera - ESPERA_EXENTA Sep/23 31:240 - Cto.:\n13467', 'UNI', '1.00', '1.100.000.0', '0.00', '1.100.000.0', '', '']
['ESPERA20IM', 'Gastos Administrativos', 'UNI', '1.00', '275.000.0', '0.00', '', '', '275.000.0']
['SUBTOTAL:', None, None, None, None, None, '1.100.000,0', '0,0', '275.000,0']
['TOTAL DE LA OPERACIÓN', None, None, None, None, None, None, None, '1.375.000']
['UN MILLON TRESCIENTOS SETENTA Y CINCO MIL GUARANIES', None, None, None, None, None, None, None, '1.375.000']
['LIQUIDACIÓN IVA: (5%) (10%) 25.000 TOTAL IVA: 25.000', None, None, None, None, None, None, None, None]


In [20]:
import os

root_dir = os.getcwd()

CantArchivos = 0

# listaf = []

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_path = os.path.join(dirpath, filename)
            # print(file_path)

# with pdfplumber.open(file_path) as temp:
#     CantArchivos += 1
#     first_page = temp.pages[0]
#     # print(first_page.extract_table())
#     tabla = first_page.extract_table()
#     encabezados = tabla[0]
#     df = pd.DataFrame(columns=encabezados)
#     if CantArchivos == 2:
#         break    # break here
                
# print(file_path)

with pdfplumber.open(file_path) as temp:
    first_page = temp.pages[0]
    print(first_page.extract_table())
    tabla = first_page.extract_table()
    # Se carga las filas del dataframe
    for fila in tabla:
        df.loc[len(df)] = fila           

                    # print("Filas cargadas: {}".format(len(df))) 
                    # df =df.drop(0)

                # # Se agrega el Nro de Factura al Df
                # texto = first_page.extract_text()
                # nroFact  = texto[0:15]
                # print("Nro Factura: {}".format(nroFact)) 
                # df['NroFactura']=nroFact

                # #Se agrega la fecha y Hora impresa en la factura
                # text = first_page.extract_text()
                # lines = text.split('\n')
                # filtered_lines = [line for line in lines if 'Fecha y hora' in line]
                # # print('\n'.join(filtered_lines))

                # str(filtered_lines)
                # fechafact =str(filtered_lines)[27:37]
                # print("Fecha Emision: {}".format(fechafact))
                # df['FechaEmision']=fechafact

                # Eliminar primera fila
        df =df.drop(0)
                # Eliminar las filas con Nulos
        df = df.dropna()

[['Cod', 'Descripcion', 'Unidad de\nmedida', 'Cantidad', 'Precio\nUnitario', 'Descuento', 'Exentas', '5%', '10%'], ['ESPERA20EX', 'Espera - ESPERA_EXENTA Nov/23 33:240 - Cto.:\n13467', 'UNI', '1.00', '1.140.000.0', '0.00', '1.140.000.0', '', ''], ['ESPERA20IM', 'Gastos Administrativos', 'UNI', '1.00', '285.000.0', '0.00', '', '', '285.000.0'], ['SUBTOTAL:', None, None, None, None, None, '1.140.000,0', '0,0', '285.000,0'], ['TOTAL DE LA OPERACIÓN', None, None, None, None, None, None, None, '1.425.000'], ['UN MILLON CUATROCIENTOS VEINTICINCO MIL GUARANIES', None, None, None, None, None, None, None, '1.425.000'], ['LIQUIDACIÓN IVA: (5%) (10%) 25.909 TOTAL IVA: 25.909', None, None, None, None, None, None, None, None]]


KeyError: '[0] not found in axis'

In [18]:
df

,Cod,Descripcion,Unidad de\nmedida,Cantidad,Precio\nUnitario,Descuento,Exentas,5%,10%
1,ESPERA20EX,Espera - ESPERA_EXENTA Nov/23 33:240 - Cto.:\n...,UNI,1.00,1.140.000.0,0.00,1.140.000.0,,
2,ESPERA20IM,Gastos Administrativos,UNI,1.00,285.000.0,0.00,,,285.000.0


In [3]:
import os

root_dir = os.getcwd()

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_path = os.path.join(dirpath, filename)
            # print(file_path)
            with pdfplumber.open(file_path) as temp:
                first_page = temp.pages[0]
                tabla = first_page.extract_table()
                encabezados = tabla[0]
                df = pd.DataFrame(columns=encabezados)

                for fila in tabla:
                    df.loc[len(df)] = fila
                print("Filas cargadas: {}".format(len(df))) 
                df =df.drop(0)


Filas cargadas: 7
Filas cargadas: 7
Filas cargadas: 7


In [4]:
encabezados

['Cod',
 'Descripcion',
 'Unidad de\nmedida',
 'Cantidad',
 'Precio\nUnitario',
 'Descuento',
 'Exentas',
 '5%',
 '10%']

In [1]:
import tabula

In [2]:
import tabula

# Leer el archivo PDF
file = "38_Septiembre_2023.pdf"
tables = tabula.read_pdf(file, pages='all')

# tables es una lista de dataframes
for table in tables:
   print(table)

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


JavaNotFoundError: `java` command is not found from this Python process.Please ensure Java is installed and PATH is set for `java`

In [ ]:
with pdfplumber.open("38_Septiembre_2023.pdf") as temp:
   first_page = temp.pages[0]
   print(first_page.extract_table())

In [ ]:
# Declaración de un dataframe vacio para la carga
import pandas as pd
tabla = first_page.extract_table()
encabezados = tabla[0]
df = pd.DataFrame(columns=encabezados)

In [ ]:
# Se carga las filas del dataframe
for fila in tabla:
    df.loc[len(df)] = fila
print("Filas cargadas: {}".format(len(df))) 
df =df.drop(0)

In [ ]:
df

In [ ]:
texto = first_page.extract_text()
nroFact  =b[0:15]
print("Nro Factura: {}".format(nroFact)) 
df['NroFactura']=nroFact

In [ ]:
text = first_page.extract_text()
lines = text.split('\n')
filtered_lines = [line for line in lines if 'Fecha y hora' in line]
# print('\n'.join(filtered_lines))

str(filtered_lines)
fechafact =str(filtered_lines)[27:37]
print("Fecha Emision: {}".format(fechafact))


In [ ]:
df['FechaEmision']=fechafact

In [ ]:
df

In [ ]:
df = df.drop(range(2, len(df), 1),axis=0)

In [ ]:
df

In [ ]:
df.dtypes